In [ ]:
import pandas as pd
import datetime
from datetime import date, time, datetime
import itertools
import numpy as np
import math
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
from statsmodels.tsa.api import VAR

import warnings
warnings.filterwarnings("ignore")

Section 1: Data Preprocessing

In [ ]:
# Read the dataset
df = pd.read_excel("../../data/market_data.xlsx") 

In [ ]:
# Remove variables that are not in the range of study
vic = df.filter(items=['Time (UTC+10)', 'Regions VIC Trading Price ($/MWh)',
                       'Regions VIC Operational Demand (MW)',
                       'Regions VIC Trading Total Intermittent Generation (MW)'])

vic = vic.rename(columns={'Time (UTC+10)': 'Time', 'Regions VIC Trading Price ($/MWh)': 'Spot Price',
                          'Regions VIC Operational Demand (MW)': 'Demand',
                          'Regions VIC Trading Total Intermittent Generation (MW)': 'Supply'})

In [ ]:
# Recast data type of datetime column to datetime type
vic['Time'] = pd.to_datetime(vic['Time'])

In [ ]:
vic_tf = vic.copy()

In [ ]:
# Perform log transformation on the attributes of interest
min_price = min(vic_tf['Spot Price'])
log_const = 1 - min_price

vic_tf['Spot Price'] = np.log(log_const + vic_tf['Spot Price'])
vic_tf['Demand'] = np.log(log_const + vic_tf['Demand'])
vic_tf['Supply'] = np.log(log_const + vic_tf['Supply'])

Section 2: Time Series Split Cross-validation for Model Selection

In [ ]:
# Retrieve records for the cross-validation period

# Dataset that only contains the spot prices
cv_vic_price = vic_tf.loc[(vic_tf['Time'].dt.date >= date(2021,1,1)) & 
                       (vic_tf['Time'].dt.date <= date(2021,6,30)), 'Spot Price']

# Dataset that contains the spot prices, demand and supply
cv_vic_full = vic_tf.loc[(vic_tf['Time'].dt.date >= date(2021,1,1)) & 
                      (vic_tf['Time'].dt.date <= date(2021,6,30))].drop(columns='Time')

The following blocks of code in this section were modified by Group 14 and the attribute is given to:
Soumya,S (2020), Cross Validation in Time Series, Medium, 
https://medium.com/@soumyachess1496/cross-validation-in-time-series-566ae4981ce4.

In [ ]:
# Perform 10 folds of cross-validation
tscv = TimeSeriesSplit(n_splits = 10)

In [ ]:
rmse = []

# Use the AR(1) model
for train_index, test_index in tscv.split(cv_vic_price):
    cv_train, cv_test = cv_vic_price.iloc[train_index].reset_index()['Spot Price'], cv_vic_price.iloc[test_index].reset_index()['Spot Price']
    
    # Fit the model
    autoReg = sm.tsa.AutoReg(cv_train, 1).fit()
    
    # Forecast the spot prices
    predictions = autoReg.predict(cv_test.index.values[0], cv_test.index.values[-1], dynamic = False)[1:]
    true_values = cv_test.values[1:]
    
    # Calculate the root mean squared error
    rmse.append(math.sqrt(mean_squared_error(true_values, predictions)))
    
print("RMSE of AR(1): {}".format(np.mean(rmse)))

In [ ]:
rmse = []

# Use the MA(1) model
for train_index, test_index in tscv.split(cv_vic_price):
    cv_train, cv_test = cv_vic_price.iloc[train_index].reset_index()['Spot Price'], \
                        cv_vic_price.iloc[test_index].reset_index()['Spot Price']
    
    # Fit the model
    arma = sm.tsa.ARMA(cv_train,(0,1)).fit()
    
    # Forecast the spot prices
    predictions = arma.predict(cv_test.index.values[0], cv_test.index.values[-1], dynamic = False)
    true_values = cv_test.values
    
    # Calculate the root mean squared error
    rmse.append(math.sqrt(mean_squared_error(true_values, predictions)))
    
print("RMSE of MA(1): {}".format(np.mean(rmse)))

In [ ]:
rmse = []

# Use the ARMA(1,1) model
for train_index, test_index in tscv.split(cv_vic_price):
    cv_train, cv_test = cv_vic_price.iloc[train_index].reset_index()['Spot Price'], \
                        cv_vic_price.iloc[test_index].reset_index()['Spot Price']
    
    # Fit the model
    arma = sm.tsa.ARMA(cv_train,(1,1)).fit()
    
    # Forecast the spot prices
    predictions = arma.predict(cv_test.index.values[0], cv_test.index.values[-1], dynamic = False)
    true_values = cv_test.values
    
    # Calculate the root mean squared error
    rmse.append(math.sqrt(mean_squared_error(true_values, predictions)))
    
print("RMSE of ARMA(1,1): {}".format(np.mean(rmse)))

In [ ]:
rmse = []

# Use the ARMAX(1,1) model
# The model incorporates spot price as the endogenous variables while demand and supply are treated as exogenous variables
for train_index, test_index in tscv.split(cv_vic_full):
    cv_train, cv_test = cv_vic_full.iloc[train_index].reset_index().drop(columns='index'), \
                        cv_vic_full.iloc[test_index].reset_index().drop(columns='index')
    
    # Fit the model
    arma = sm.tsa.ARMA(cv_train['Spot Price'],(1,1), exog=np.array(cv_train['Supply'],cv_train['Demand'])).fit()
    
    # Forecast the spot prices
    predictions = arma.predict(cv_test.index.values[0], cv_test.index.values[-1], dynamic = False)
    true_values = cv_test['Spot Price'].values
    
    # Calculate the root mean squared error
    rmse.append(math.sqrt(mean_squared_error(true_values, predictions)))
    
print("RMSE of ARMAX(1,1): {}".format(np.mean(rmse)))

In [ ]:
rmse = []

# Use the vector autoregressive model, VAR(1)
# The model incorporates spot price, demand and supply as endogenous variables
for train_index, test_index in tscv.split(cv_vic_full):
    cv_train, cv_test = cv_vic_full.iloc[train_index].reset_index().drop(columns='index'), \
                        cv_vic_full.iloc[test_index].reset_index().drop(columns='index')
    
    var = VAR(cv_train)
    
    # Fit the model
    results = var.fit(1)
    lag_order = results.k_ar
    
    # Forecast the spot prices
    predictions = results.forecast(cv_test.values[-lag_order:], steps = len(cv_test))
    true_values = cv_test.values
    
    # Calculate the root mean squared error
    rmse.append(math.sqrt(mean_squared_error(true_values, predictions)))
    
print("RMSE of VAR(1): {}".format(np.mean(rmse)))

Section 3: Fit and Forecast Using the Actual Training and Test Set with MA(1)

In [ ]:
# Retrieve records for the training and test period
train_df = vic_tf.loc[vic_tf['Time'].dt.date <= date(2020,12,31)].reset_index()['Spot Price']
test_df = vic_tf.loc[vic_tf['Time'].dt.date >= date(2021,6,30)].reset_index()['Spot Price']

In [ ]:
# Fit MA(1) with the training set
arma = sm.tsa.ARMA(train_df,(0,1)).fit()

# Forecast the spot prices for the test set with the fitted model
predictions = arma.predict(test_df.index.values[0], test_df.index.values[-1], dynamic = False)
true_values = test_df.values

Section 4: Run the Weighted Future Average Algorithm (as in Mandatory Task but with Spot Price Predictions)

In [ ]:
vic = vic.drop(columns=['Demand', 'Supply']).rename(columns={'Spot Price': 'Spot Price Prediction'})

In [ ]:
# Obtain the forecasted spot prices in actual scale 
vic.loc[len(vic)-len(predictions):,'Spot Price Prediction'] = (np.exp(predictions)-677.37)[:, None]

In [ ]:
# Battery properties
BATTERY_POWER = 300
BATTERY_CAP = 580
CHARGE_EFF = 90
DISCHARGE_EFF = 90
MLF = 0.991

In [ ]:
def raw_power(charge_forecast, discharge_forecast, opening_cap):
    '''Takes in the forecasted battery behaviour and opening capacity, returns the amount of raw power.'''
    
    if charge_forecast == 1:
        return -min(BATTERY_POWER, (BATTERY_CAP - opening_cap) / (CHARGE_EFF / 100) * 2) 
    elif discharge_forecast == 1:
        return min(BATTERY_POWER, opening_cap * 2)
    else:
        return 0

In [ ]:
def market_dispatch(raw_power):
    '''Takes in the raw power, returns the power for market dispatch.'''
    
    if raw_power < 0:
        return raw_power / 2
    elif raw_power > 0:
        return (raw_power / 2) * DISCHARGE_EFF / 100
    else:
        return 0 

In [ ]:
def market_revenue(market_dispatch, spot_price):
    '''Takes in the power for market dispatch and spot price, returns the market revenue generated.'''
    
    if market_dispatch < 0:
        return market_dispatch * spot_price * (1 / MLF)
    elif market_dispatch > 0:
        return market_dispatch * spot_price * MLF
    else:
        return 0

In [ ]:
def closing_capacity(market_dispatch, opening_cap):
    '''Takes in the power for market dispatch and opening capacity, returns the closing capacity.'''
    
    if market_dispatch < 0:
        closing_cap_cand = opening_cap - market_dispatch * (CHARGE_EFF / 100)
        return max(0, min(closing_cap_cand, BATTERY_CAP))
    elif market_dispatch >= 0:
        closing_cap_cand = opening_cap - market_dispatch * (100 / DISCHARGE_EFF)
        return max(0, min(closing_cap_cand, BATTERY_CAP))
    else:
        return 0

In [ ]:
def weighted_future_avg(df, index, num_future_periods):
    '''Takes in the full set of spot prices and index of the current period, 
    returns the weighted future average price relative to the current period.'''
    
    total_periods = len(df)
    
    # Compute the weighted future average price relative to periods with at least 10 future periods
    if index < (total_periods - num_future_periods):   
        future_df = df.loc[(index + 1):(index + num_future_periods), "Spot Price Prediction"].to_frame()
        future_df["Weights"] = list(range(num_future_periods, 0, -1))
        future_avg = round(np.average(future_df["Spot Price Prediction"], weights = future_df["Weights"]),2)
        
    # Compute the weighted future average price relative to periods with less than 10 future periods, excluding the last period
    elif (index >= (total_periods - num_future_periods)) and (index != (total_periods - 1)):
        future_df = df.loc[(index + 1):total_periods, "Spot Price Prediction"].to_frame()
        future_df["Weights"] = list(range(num_future_periods, num_future_periods - (total_periods - index) + 1, -1))
        future_avg = round(np.average(future_df["Spot Price Prediction"], weights = future_df["Weights"]),2)
    
    # Set the weighted future average price of the last period as its spot price
    elif index == (total_periods - 1):
        future_avg = df.loc[index, "Spot Price Prediction"]
        
    else:
        future_avg = 0
        
    return future_avg

In [ ]:
def battery_forecast(df, index, comparison_threshold):
    '''Sets the forecasted charge and discharge behaviour of the current period.'''
    
    current_price = df.loc[index, "Spot Price Prediction"]
    weighted_avg_future = df.loc[index, "Future Average"]
    
    # Calculate the absolute difference between the current price and weighted future average price
    current_future_diff = abs(weighted_avg_future - current_price)
    
    # Determine the discharge behaviour of the current period
    if (current_price > weighted_avg_future) and (current_future_diff >= comparison_threshold):
        df["Discharge Forecast"][index] = 1
    else:
        df["Discharge Forecast"][index] = 0
        
    # Determine the charge behaviour of the current period
    if (current_price < weighted_avg_future) and (current_future_diff >= comparison_threshold):
            df["Charge Forecast"][index] = 1       
    else:
        df["Charge Forecast"][index] = 0
        
    return

In [ ]:
# Categorise the technical variables according to data type
floats_vars = ["Future Average", "Raw Power", "Market Dispatch", "Market Revenue", "Opening Capacity", "Closing Capacity"]
ints_vars = ["Charge Forecast", "Discharge Forecast"]

In [ ]:
def battery(df, threshold, num_future_periods):
    '''Create a dataframe with all the technical variables.'''
    
    # Initialise all entries as 0
    for var in ints_vars:
        df[var] = 0
    for var in floats_vars:
        df[var] = 0.0
    
    # Update the values of the technical variables in each period
    for index, row in df.iterrows():
        # Computed the weighted future average price 
        future_avg = weighted_future_avg(df, index, num_future_periods)
        df["Future Average"][index] = future_avg
        
        # Forecast the charge and discharge behaviour of the battery
        battery_forecast(df, index, threshold)
        
        # Set the opening capacity of the current period as the previous period's closing capacity, 
        # excluding the first period which is assumed to start discharged
        if index != 0:
            df["Opening Capacity"][index] = df.loc[index-1, "Closing Capacity"]
        
        df["Raw Power"][index] = raw_power(df["Charge Forecast"][index], df["Discharge Forecast"][index], df["Opening Capacity"][index])
        df["Market Dispatch"][index] = market_dispatch(df["Raw Power"][index])
        df["Market Revenue"][index] = market_revenue(df["Market Dispatch"][index], df["Spot Price Prediction"][index])
        df["Closing Capacity"][index] = closing_capacity(df["Market Dispatch"][index], df["Opening Capacity"][index])
        
    return

In [ ]:
# Optimal number of future periods and optimal comparison threshold by tuning using the cross-validation period
# This has been done in the notebook named "Mandatory Task Approach"
opt_future_periods = 10
opt_threshold = 6.0

# Create the dataframe with technical variables for the full sample period
battery(vic, opt_threshold, opt_future_periods)

In [ ]:
# Retrieve the index of the first period in the test set
index_test = vic.index[vic["Time"] == datetime(2021, 7, 1, 0, 0, 0)][0]

test = vic.iloc[index_test:,]

# Calculate the predicted market revenue for respective periods
print("Total Revenue for Test Period: " + str(round(sum(test["Market Revenue"]),2)))
print("Total Revenue for Full Period: " + str(round(sum(vic["Market Revenue"]),2)))

Section 5: Generate Output Files

In [ ]:
# Create output dataframe used for submission
output = vic[["Time", "Raw Power", "Spot Price Prediction", "Opening Capacity"]]. \
         rename(columns={"Time":"datetime", "Raw Power":"power", "Spot Price Prediction":"spot price prediction", "Opening Capacity":"capacity"})

In [ ]:
# Create submission files
output.to_csv("../../results/bonus_submission.csv", index=False)